In [1]:
from keys import load_api_keys
load_api_keys()

In [2]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a word to replace the following: {word}?"

# Set the "OPENAI_API_KEY" environment variable before running following line.
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)


/tmp/ipykernel_70688/1895198210.py:6: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)
/tmp/ipykernel_70688/1895198210.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


In [3]:
llm_chain("artificial")

/tmp/ipykernel_70688/1842958972.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_chain("artificial")


{'word': 'artificial', 'text': '\n\nSynthetic'}

In [4]:
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSynthetic'},
 {'text': '\n\nCleverness'},
 {'text': '\n\nandroid'}]

In [7]:
prompt_template = "Looking at the context of '{context}'. What is an appropriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="fan", context="inanimate object")
# or llm_chain.run(word="fan", context="object")

'\n\nappliance'

In [8]:
llm_chain.predict(word="fan", context="humans")

'\n\nSupporter'

In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, output_parser=output_parser, input_variables=[]),
    output_parser=output_parser)

llm_chain.predict()

['synthetic',
 'man-made',
 'fake',
 'imitation',
 'simulated',
 'faux',
 'fabricated',
 'counterfeit',
 'ersatz',
 'pseudo',
 'false',
 'manufactured',
 'unnatural',
 'contrived',
 'pretend',
 'mock',
 'spurious',
 'bogus',
 'phony']

In [10]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation.predict(input="List all Age of Empires military units by how powerful they are.")

/tmp/ipykernel_70688/1335568861.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
/tmp/ipykernel_70688/1335568861.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


' Sure, there are a few different ways to rank the Age of Empires military units by power. One way is to look at their attack and defense stats. In that case, the most powerful units would be the Cataphract, War Elephant, and Elite Cataphract, all of which have high attack and defense stats. The next tier would include units like the Elite War Elephant, Elite Cataphract, and Elite Teutonic Knight. These units have slightly lower attack and defense stats, but are still very strong. Moving down the list, we have units like the Elite Samurai, Elite Mameluke, and Elite War Wagon, which have good attack stats but slightly lower defense. Finally, at the bottom of the list, we have units like the Elite Longbowman, Elite Chu Ko Nu, and Elite Janissary, which have high attack but lower defense. Of course, this is just one way to rank the units by power. Another way could be to consider their cost and training time, as well as their abilities and bonuses against certain units. It really depends 

In [11]:
conversation.predict(input="And the next 4?")

' The next four units on the list would be the Elite Jaguar Warrior, Elite Berserk, Elite War Elephant, and Elite Teutonic Knight. These units have good attack and defense stats, but may have specific bonuses or abilities that make them more powerful in certain situations. For example, the Elite Jaguar Warrior has a bonus against infantry units, while the Elite Berserk has a regeneration ability. The Elite War Elephant and Elite Teutonic Knight both have high attack and defense stats, but the War Elephant is slower and more expensive to train, while the Teutonic Knight has a bonus against cavalry units. Again, it really depends on the specific situation and strategy being used in the game.'

In [12]:
# poet
poet_template: str = """You are an American poet, your job is to come up with\
poems based on a given theme.

Here is the theme you have been asked to generate a poem on:
{input}\
"""

poet_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["input"], template=poet_template)

# creating the poet chain
poet_chain: LLMChain = LLMChain(
    llm=llm, output_key="poem", prompt=poet_prompt_template)

# critic
critic_template: str = """You are a critic of poems, you are tasked\
to inspect the themes of poems. Identify whether the poem includes romantic expressions or descriptions of nature.

Your response should be in the following format, as a Python Dictionary.
poem: this should be the poem you received 
Romantic_expressions: True or False
Nature_descriptions: True or False

Here is the poem submitted to you:
{poem}\
"""

critic_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["poem"], template=critic_template)

# creating the critic chain
critic_chain: LLMChain = LLMChain(
    llm=llm, output_key="critic_verified", prompt=critic_prompt_template)

In [13]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[poet_chain, critic_chain])

In [15]:
# Run the poet and critic chain with a specific theme
theme: str = "the beauty of nature and love"
review = overall_chain.run(theme)

# Print the review to see the critic's evaluation
print(review)



poem: Nature's beauty surrounds us,
In every tree and flower,
A love that's pure and boundless,
In every passing hour.

The sun sets in a blaze of colors,
As the birds sing their sweet tune,
A love that's everlasting,
In the light of the moon.

The gentle breeze whispers secrets,
Through the rustling leaves,
A love that's ever-changing,
But always believes.

The mountains stand tall and mighty,
As the rivers flow below,
A love that's strong and steadfast,
Through every ebb and flow.

The flowers bloom in vibrant hues,
As the bees dance around,
A love that's full of wonder,
In every sight and sound.

Nature's beauty and love intertwined,
In a perfect harmony,
A reminder of the magic,
That surrounds us constantly.

So let us take a moment,
To appreciate and cherish,
The beauty of nature and love,
That will never perish.
Romantic_expressions: True
Nature_descriptions: True


Custom chain

In [16]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [17]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Artificial means something that is made or produced by humans, rather than occurring naturally. It can also refer to something that is not genuine or authentic, but rather created or imitated to appear real.

Synthetic
